# 簡單工廠 Simple Factory 

通過專門定義一個類（工廠）來創建其他類的實例，被創建的實例通常都有共同的父類

## Exercise 1  計算機


- 練習：做個簡易計算機，輸入兩個數字和運算符號，得到結果。
- 想法：為了方便取用且降低耦合性以防止牽一髮動全身，可讓介面取用「運算子」，做一個運算的superclass，而使用多態性，讓介面取用時實例化出合適的運算子，返回結果。

In [16]:
# 定義一個抽象父類別
class Operation(object):

    # 定義屬性，給定default值
    def __init__(self, numberA=0, numberB=0):
        self.numberA = numberA
        self.numberB = numberB

    # 定義出一個抽象方法
    def getResult(self):
        pass


# 定義各個可以實例化的生產線子類別（可使用父類屬性）
# 加法
class OperationAdd(Operation):
    def getResult(self):
        result = self.numberA + self.numberB
        return result

# 減法
class OperationSub(Operation):
    def getResult(self):
        result = self.numberA - self.numberB
        return result

# 乘法
class OperationMul(Operation):
    def getResult(self):
        result = self.numberA * self.numberB
        return result

# 除法
class OperationDiv(Operation):
    def getResult(self):
        if self.numberB == 0:
            print('除數不得為0')
        result = self.numberA / self.numberB
        return result


# 定義一個簡單運算工廠類別 - 做一個工廠，以後可以新增及移除生產線，也可以選擇要用那個生產線製造東西, 也就是去選擇要去實例化哪個object
class OperationFactory(object):

    def createOperate(self, oper):
        if oper == '+':
            return OperationAdd()
        elif oper == '-':
            return OperationSub()
        elif oper == '*':
            return OperationMul()
        elif oper == '/':
            return OperationDiv()


In [27]:
# client side
# if__name__ 這個其實可以不用寫在jupyer notebook，這個代表.py被直接執行時，寫在這下面的程式碼將被執行，而當.py檔以模組形式被其他檔案匯入時，這之下的程式碼不會被執行。

if __name__ == '__main__':
    op = OperationFactory()
    plus = op.createOperate('+')
    plus.numberA = 3455
    plus.numberB = 2222
    result = plus.getResult()
    print(result)

5677


# 策略模式 Strategy Pattern

 - 定義了算法family，分別封裝起來，使他們之間可以互相替換，此模式使算法變化時，不會影響到使用算法的客戶
 - 策略模式是一種定義一系列算法的方法，從概念上來看，所有這些算法完成的都是相同的工作，只是實現過程不同，它可以用相同的方式調用所有算法，減少了各種算法類與使用算法類的耦合。
 - 策略模式的其中一個優點是可以簡化單元測試，因為每個算法都有自己的類，可以通過自己的接口單獨測試。


## Exercise 2 收銀台
- 練習：建一個收銀程式，當輸入價格及數量時，可以返回收銀價格。而這個程式可加裝各種促銷模式及方法(例：打8折、滿千送百之類的)


### 想法一 - 簡單工廠

- 先設置一個現金收費的抽象父類，參數為原價，返回各種打折之類的算完後的價格
- 設置各個優惠策略的具體子類，繼承前面的現金收費抽象類
- 做一個獨立的現金收費工廠類，參數為優惠策略的字串，由字串去決定要實例化哪個類
- 客戶端：給定單價、數量、優惠，然後得出合計值

In [1]:
import math
# 設置一個優惠策略的父類

class CashSuper(object):

    def __init__ (self):
        pass


    # 建立一個抽象方法
    def acceptCash(self, money):
        return money


# 再來設置各個優惠策略的子類
# 正常收費
class CashNormal(CashSuper):
    def acceptCash(self, money):
        return super().acceptCash(money)

# 打折收費
class CashRebate(CashSuper):
    # 初始化時給定屬性
    def __init__(self, moneyRebate):
        self.moneyRebate = moneyRebate

    # 改寫方法
    def acceptCash(self, money):
        return super().acceptCash(money) * self.moneyRebate

# 現金回饋收費
class CashReturn(CashSuper):
    # 初始化
    def __init__(self, moneyCondition, moneyReturn):
        self.moneyCondition = moneyCondition
        self.moneyReturn = moneyReturn 
    
    # 改寫方法
    def acceptCash(self, money):
        money = super().acceptCash(money)
        if money >= self.moneyCondition:
            result = money - math.floor(money / self.moneyCondition) * self.moneyReturn
        return result


# 現金收費工廠

class CashFactory(object):
    
    def createCashAccept(self, cashAccept):
        if cashAccept == '正常收費':
            return CashNormal()
        elif cashAccept == '滿千送百':
            return CashReturn(1000, 100)
        elif cashAccept == '打8折':
            return CashRebate(0.8)

In [26]:
# client side

if __name__ == '__main__':

    # 設定價格、數量、優惠方法
    price = 3600
    num = 2
    accept_method = '滿千送百'

    # 製造算法的工廠
    cs = CashFactory()
    cs_method = cs.createCashAccept(accept_method)

    # 取用算法後，執行他的方法
    total_price = price * num
    total = cs_method.acceptCash(total_price)

    print(f'商品價格：{price}\n商品數量：{num}\n優惠方式：{accept_method}\n優惠後價格：{total}')


商品價格：3600
商品數量：2
優惠方式：滿千送百
優惠後價格：6500


### 想法二 - 策略模式

- 原先簡單工廠模式的問題：如果更改打折折數或現金回饋條件的時候, 就要去修改工廠的code，然後程式就要再重新部署，不夠方便
- 改寫完還是有點搞不懂比簡單工廠好在哪...我錯了QQ
- 這篇的憤怒鳥例子比較好懂：https://medium.com/@a4793706/design-patterns-python-%E4%B8%80-%E7%AD%96%E7%95%A5%E6%A8%A1%E5%BC%8F-2f9b4bcaa20

In [2]:
import math
# 重構為策略模式

# 設置一個優惠策略的父類(抽象算法的父類)
class CashSuper(object):

    def __init__ (self):
        pass

    # 建立一個抽象方法
    def acceptCash(self, money):
        return money


# 再來設置各個優惠策略的子類（具體的策略）
# 正常收費
class CashNormal(CashSuper):
    def acceptCash(self, money):
        return super().acceptCash(money)

# 打折收費
class CashRebate(CashSuper):
    # 初始化時給定屬性
    def __init__(self, moneyRebate):
        self.moneyRebate = moneyRebate

    # 改寫方法
    def acceptCash(self, money):
        return super().acceptCash(money) * self.moneyRebate

# 現金回饋收費
class CashReturn(CashSuper):
    # 初始化
    def __init__(self, moneyCondition, moneyReturn):
        self.moneyCondition = moneyCondition
        self.moneyReturn = moneyReturn 
    
    # 改寫方法
    def acceptCash(self, money):
        money = super().acceptCash(money)
        if money >= self.moneyCondition:
            result = money - math.floor(money / self.moneyCondition) * self.moneyReturn
        return result



In [3]:
# 定義一個封裝這些算法策略的類
class Context(object):

    # 傳入具體的策略對象
    def __init__(self, strategy):
        self.strategy = strategy

    # 調用算法的方法
    def getResult(self, money):
        if self.strategy == '正常收費':
            cs = CashNormal()
        elif self.strategy == '滿千送百':
            cs = CashReturn(1000, 100)
        elif self.strategy == '打8折':
            cs = CashRebate(0.8)

        return cs.acceptCash(money)


In [5]:
# client side

if __name__ == '__main__':

    # 設定價格、數量、優惠方法
    price = 3600
    num = 2
    accept_method = '打8折'

    # 製造算法的工廠
    cs = Context(accept_method)
    

    # 取用算法後，執行他的方法
    total_price = price * num
    total = cs.getResult(total_price)

    print(f'商品價格：{price}\n商品數量：{num}\n優惠方式：{accept_method}\n優惠後價格：{total}')


商品價格：3600
商品數量：2
優惠方式：打8折
優惠後價格：5760.0
